In [1]:
using Serialization, Distributions, AffineInvariantMCMC, StableRNGs 
include("ode_problem.jl")
include("target_probability.jl");

### Name Run

run: 202  
test data: active G protein timecourse and dose response  
augmented: yes  
weighed by AF3 confidence: yes   
prior offset: +/- 3  
binding prior offset: +/- 1  

In [2]:
run = "202"

"202"

Define Prior Distributions

In [3]:
#generate bounds for prior
bound_offset = [3,3]
binding_bound_offset = [1,1]
prior_bounds = return_prior_bounds_empirical(bound_offset, binding_bound_offset) #for target probability
prior_distributions = [Uniform(prior_bounds[i]...) for i in 1:length(prior_bounds)];

Define Test/Train Data

In [4]:
#first input we need are data points and each point's associated standard deviation to evaluate the likelihood
experiment_timecourse = deserialize("outputs/000_processed_active_g_timecourse.dict")
experiment_dose_response = deserialize("outputs/000_processed_active_g_dose_response.dict")

data_points = cat(experiment_timecourse["response"], experiment_dose_response["response"], dims=1 )
std_dev = cat(fill(experiment_timecourse["average_error"], length(experiment_timecourse["response"])), fill(experiment_dose_response["average_error"], length(experiment_dose_response["response"])), dims=1)

#load different ligand dosages needed to simulate all training data
normalize_to = normalize_to_dose()
ligand_doses = cat(return_ligand_dose_nM(),normalize_to, dims=1); #normalize to last stimulation

Define ODEProblems for different intial conditions

In [5]:
#next input we need is an ode problem
odeproblems = []
for l in ligand_doses
    odesys, u0, tspan, p = return_ode_problem_default_inputs(l)
    if l != 1000.0
        tspan = (0,65)
    end
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p) #note p will be overwritten in first run of mcmc algorithm
    push!(odeproblems, prob)
end

Define ODE Solver Inputs

In [6]:
#and ode solver inputs, save at changes based on timecourse vs dose response output
odesolver_inputs_timecourse = return_ode_problem_solver_default_inputs("timecourse")
odesolver_inputs_dose_response = return_ode_problem_solver_default_inputs("dose response");

Define Type of ODESolution

In [7]:
#get type of ODEProblem solution
pred = DifferentialEquations.solve(odeproblems[1], odesolver_inputs_dose_response["solver"], abstol=odesolver_inputs_dose_response["abstol"], reltol=odesolver_inputs_dose_response["reltol"], saveat=odesolver_inputs_dose_response["saveat"])
ode_sol_type = typeof(pred);

Define Augmented Likelihood Term

In [8]:
#also need hyperparameters for augmenting likelihood
augmented_likelihood_parameters = deserialize("outputs/000_regularization_parameters_predicted.dict");

Define Target Distribution for Sampling

In [9]:
target_probability = parameters -> logprob_augmented(parameters, prior_distributions, data_points, std_dev, odeproblems, odesolver_inputs_timecourse, odesolver_inputs_dose_response, ode_sol_type, augmented_likelihood_parameters)

#31 (generic function with 1 method)

Affine Invariant Sampler Hyperparameters

In [ ]:
#affine invariant sampler hyperparameters
numdims = 8
numwalkers = 1000
thinning = 1
numsamples_perwalker = 1000
burnin = 1000
n_subchains = 50
a_burnin = 4
a = 2
rng = StableRNG(28475);

Run Affine Invariant Sampler from 0

In [11]:
x0 = Matrix{Float64}(undef,numdims,numwalkers)
for i in 1:numwalkers
    x0[:,i] = [rand(rng, prior_distributions[j]) for j in 1:numdims]
end

chain0, llhoodvals0 = AffineInvariantMCMC.sample(target_probability, numwalkers, x0, burnin, thinning, a_burnin, rng=rng)
serialize("outputs/$(run)_posterior_samples_subchain_0.jls", chain0)
serialize("outputs/$(run)_posterior_samples_likelihood_subchain_0.jls", llhoodvals0)

#release from RAM
chain0 = nothing
llhoodvals0 = nothing

for i in 1:n_subchains
    chain = deserialize("outputs/$(run)_posterior_samples_subchain_$(i-1).jls")
    chain, llhoodvals = AffineInvariantMCMC.sample(target_probability, numwalkers, chain[:, :, end], numsamples_perwalker, thinning, a, rng=rng)
    serialize("outputs/$(run)_posterior_samples_subchain_$(i).jls", chain)
    serialize("outputs/$(run)_posterior_samples_likelihood_subchain_$(i).jls", llhoodvals)
end

┌ Warning: At t=58.0134782274667, dt was forced below floating point epsilon 7.105427357601002e-15, and step error estimate = 1.0197237252787326. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/Zrnkf/src/integrator_interface.jl:623
Progress:   0%|▏                                        |  ETA: 1:08:20┌ Warning: At t=453.8085226191582, dt was forced below floating point epsilon 5.684341886080802e-14, and step error estimate = 1.019725754902985. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/Zrnkf/src/integrator_interface.jl:623
Progress: 100%|█████████████████████████████████████████| Time: 0:51:43
Progress: 100%|█████████████████████████████████████████| Time:

Run Affine Invariate Sampler from N Subchain

In [12]:
#start = 
#stop = 
#for i in start:stop
    #chain = deserialize("outputs/$(run)_posterior_samples_subchain_$(i-1).jls")
    #chain, llhoodvals = AffineInvariantMCMC.sample(target_probability, numwalkers, chain[:, :, end], numsamples_perwalker, thinning, a, rng=rng)
    #serialize("outputs/$(run)_posterior_samples_subchain_$(i).jls", chain)
    #serialize("outputs/$(run)_posterior_samples_likelihood_subchain_$(i).jls", llhoodvals)
#end